In [ ]:
import pandas as pd
import re
import unicodedata

# Read the CSV file and create a DataFrame
df = pd.read_csv("../data/Mail/Important/messages.csv")

# Add these headers to the DataFrame, Subject, From, To, Date, Attachments, and Body
df.columns = ["Subject", "From", "To", "Date", "Attachments", "Size", "Body"]

# Display the dataset
df.head()

In [ ]:
# Remove the Size and Attachments column
df = df.drop(columns=["Size", "Attachments"])

# Convert the from and to columns to a string
df["From"] = df["From"].astype(str)
df["To"] = df["To"].astype(str)

# Normalize unicode characters
df["Body"] = df["Body"].apply(
    lambda x: unicodedata.normalize("NFKD", x)
    .encode("ascii", "ignore")
    .decode("utf-8", "ignore")
)

# Remove non-ascii characters
df["Body"] = df["Body"].apply(lambda x: re.sub(r"[^\x00-\x7F]+", "", x))

In [ ]:
# Remove leading and trailing spaces around quotes in the Subject column
df["Subject"] = df["Subject"].str.replace(r'\s*[\'"]\s*', "", regex=True)

# Remove single and double quotes from Subject, To and From columns
df["Subject"] = df["Subject"].str.replace(r'[\"\'\\"]', "", regex=True)

# Remove any of the following from the Subject column: "Fw:", "Re:", "Fwd:", "FW:", "RE:", "FWD:", "FW:" (with or without quotes)
df["Subject"] = df["Subject"].str.replace(
    r"^['\"]?(fw|re|fwd): ['\"]?", "", case=False, regex=True
)

In [ ]:
# Convert A9414643DE8E41A5B0E7C7B45FC02132-C to cody@tmvsocial.com
df["From"] = df["From"].str.replace(
    "<A9414643DE8E41A5B0E7C7B45FC02132-C", "<cody@tmvsocial.com>"
)
df["From"] = df["From"].str.replace(
    "<a9414643de8e41a5b0e7c7b45fc02132-C", "<cody@tmvsocial.com>"
)

# Convert cody@themastermindvision.com to cody@tmvsocial.com
df["From"] = df["From"].str.replace("themastermindvision.com", "tmvsocial.com")
df["To"] = df["To"].str.replace("themastermindvision.com", "tmvsocial.com")

# Transform the email addresses inside of <> in From and To columns to lowercase
df["From"] = df["From"].apply(
    lambda x: re.sub(r"<[^>]+>", lambda x: x.group().lower(), x)
)
df["To"] = df["To"].apply(lambda x: re.sub(r"<[^>]+>", lambda x: x.group().lower(), x))

# Remove leading and trailing spaces around quotes in To and From columns
df["To"] = df["To"].str.replace(r'\s*[\'"]\s*', "", regex=True)
df["From"] = df["From"].str.replace(r'\s*[\'"]\s*', "", regex=True)

# Remove single and double quotes from Subject, To and From columns
df["To"] = df["To"].str.replace(r'[\"\'\\"]', "", regex=True)
df["From"] = df["From"].str.replace(r'[\"\'\\"]', "", regex=True)

In [ ]:
# Remove any HTML tags from the 'Body' column
tags_to_remove = ["script", "style", "img", "a", "head"]
for tag in tags_to_remove:
    df["Body"] = df["Body"].str.replace(rf"<{tag}[^>]*>.*?</{tag}>", "", regex=True)

df["Body"] = df["Body"].str.replace("themastermindvision.com", "tmvsocial.com")

# remove the entire line if it starts with "Subject:, From:, Date:, To:, CC:"
df["Body"] = df["Body"].apply(
    lambda email: re.sub(
        r"^(Subject:|From:|Date:|To:|CC:).*\r\n.*\r\n", "", email, flags=re.MULTILINE
    )
)

# Remove successive >> characters if there is more than one in a row
df["Body"] = df["Body"].apply(lambda email: re.sub(r">>+", "", email))

# Remove the following " > " from the 'Body' column
df["Body"] = df["Body"].str.replace(r" > ", "", regex=True)

# Remove successive spaces and white spaces
df["Body"] = df["Body"].apply(lambda email: re.sub(r"\s+", " ", email))

# Remove underline from Body column
df["Body"] = df["Body"].str.replace(r"[_]", "", regex=True)

# Remove forwarded and original message lines from the Body column
df["Body"] = df["Body"].str.replace(
    r"[-]* forwarded message:? [-]*|[-]* original message:? [-]*",
    "",
    flags=re.IGNORECASE,
    regex=True,
)

signatures = [
    "Cody Tucker (308) 293-8040",
    "Cody Tucker 903-456-5619",
    "Cody Tucker (903) 456-5619",
    "Cody W Tucker Marketing Expert Cell: 903-456-5619 Check our Social Media Management Plans aHERE LinkedIn, TMV-Social FB TMV - Social 5400 Coal Chute Rd Kearney, NE 68847 Unsubscribe",
    "Cody W Tucker Marketing Expert Cell: 903-456-5619 Check our Social Media Management Plans HERE LinkedIn, TMV-Social FB TMV - Social 5400 Coal Chute Rd Kearney, NE 68847 Unsubscribe",
    "Cody Tucker (308) 224-2800",
]


def remove_patterns(df, column, patterns):
    for pattern in patterns:
        df[column] = df[column].str.replace(pattern, "", regex=True)
    return df


patterns = [
    "|".join(re.escape(sig) for sig in signatures),
    r"Todd E\. Hoins.*deleting it from your computer\.",
    r"A person smiling for the camera Description automatically generated",
    r"signature\d+.*Description automatically generated",
]

df = remove_patterns(df, "Body", patterns)

df

In [ ]:
# Group by 'Subject', 'From', 'To', and 'Date' and find the index of the row with the most characters in 'Body' for each group
idx = df.groupby(["Subject", "From", "To", "Date"])["Body"].apply(
    lambda x: x.str.len().idxmax(skipna=True)
)

# Filter the DataFrame to keep only the rows with these indices
df_chains = df.loc[df.index.isin(idx)]

# Remove all duplicates from the original DataFrame, including the first occurrence
df_no_chains = df.drop_duplicates(subset=["Subject", "From", "To", "Date"], keep=False)

# Concatenate the DataFrame with no chains and the DataFrame with chains
df_combined = pd.concat([df_no_chains, df_chains])

# Remove duplicates from df_combined
df_combined = df_combined.drop_duplicates(subset=["Subject", "From", "To", "Date"])

# Check the number of unique 'Subject', 'From', and 'To' combinations
assert (
    df.groupby(["Subject", "From", "To"]).ngroups
    == df_combined.groupby(["Subject", "From", "To"]).ngroups
)

# Check the total number of rows
assert df.shape[0] >= df_combined.shape[0]

df_combined

In [ ]:
# Check for duplicates in the 'Subject', 'From', 'To' and 'Date' columns
df_combined[
    df_combined.duplicated(subset=["Subject", "From", "To", "Date"], keep=False)
]

In [ ]:
# Save the cleaned data to a new CSV file
df_combined.to_csv("~/Documents/Mail/Important/cleaned_messages.csv", index=False)